In [13]:
import jams
import os
import pandas as pd
import re

In [36]:
notes_to_scale_degrees = {
    "C": "1",
    "C#": "#1",
    "Db": "b2",
    "D": "2",
    "D#": "#2",
    "Eb": "b3",
    "E": "3",
    "F": "4",
    "F#": "#4",
    "Gb": "b5",
    "G": "5",
    "G#": "#5",
    "Ab": "b6",
    "A": "6",
    "A#": "#6",
    "Bb": "b7",
    "B": "7",
}


def convert_chord_format(chord_str):
    """Convert chord to JAMS-compatible format."""
    if chord_str in ["N", "X"]:
        return chord_str

    # Regex for diminished (dim) and diminished 7th (dim7) chords
    dim_pattern = r"([A-Ga-g#]+):\((bb3|b3),\s*(b5)\)"
    dim7_pattern = r"([A-Ga-g#]+):\((bb3|b3),\s*(b5),\s*(bb7)\)"
    maj7_pattern = r"([A-Ga-g#]+):\((3),\s*(b5),\s*(b7)\)"

    # Check if the chord matches the diminished pattern (bb3, b5)
    if re.match(dim_pattern, chord_str):
        root = re.match(dim_pattern, chord_str).group(1)
        return f"{root}:dim"

    # Check if the chord matches the diminished 7th pattern (bb3, b5, bb7)
    elif re.match(dim7_pattern, chord_str):
        root = re.match(dim7_pattern, chord_str).group(1)
        return f"{root}:dim7"

    # Check if the chord matches the major 7th pattern (3, b5, b7)
    elif re.match(maj7_pattern, chord_str):
        root = re.match(maj7_pattern, chord_str).group(1)
        return f"{root}:maj7"

    # Handle inversions (chords with slash notation)
    if "/" in chord_str:
        return chord_str.split("/")[0]

    if ":(" in chord_str:
        chord_str = chord_str.split(":")[0]

    return chord_str

In [ ]:
def csv_to_jams(csv_file_path, jams_file_path):
    """
    Converts a CSV file to a JAMS file, keeping only the shorthand column.

    Parameters:
        csv_file_path (str): Path to the input CSV file.
        jams_file_path (str): Path to save the output JAMS file.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path, delimiter=";")

    # Create a new JAMS object
    jam = jams.JAMS()

    # Set the file_metadata duration to 0
    jam.file_metadata.duration = 0

    # Create a new annotation for chords
    chord_annotation = jams.Annotation(namespace="chord")

    # Iterate through the rows of the DataFrame
    for _, row in df.iterrows():
        chord_annotation.append(
            time=row["start"],
            duration=row["end"] - row["start"],
            value=convert_chord_format(row["shorthand"]),
            confidence=1.0,
        )

    # Add the annotation to the JAMS object
    jam.annotations.append(chord_annotation)

    # Save the JAMS file
    jam.save(jams_file_path)

In [40]:
dataset_path = "/Users/theo/School/2/MIR/final-project/datasets/Beethoven_Piano_Sonata_Dataset_v2/2_Annotations/ann_audio_chord"

for file in os.listdir(dataset_path):
    if file.endswith(".csv"):
        print(f"Converting {file}...")
        csv_file_path = os.path.join(dataset_path, file)
        jams_file_path = os.path.join(
            dataset_path, file.replace(".csv", ".jams")
        )
        csv_to_jams(csv_file_path, jams_file_path)
        print(f"Converted {csv_file_path} to {jams_file_path}")

Converting Beethoven_Op053-01_VA81.csv...
C:maj
D:7/C
G:maj/B
Bb:maj
C:7/Bb
F:maj/A
F:min/Ab
G:7
D:hdim7/Ab
G:7
D:hdim7/Ab
G:7
D:hdim7/Ab
G:7
C:min
G:maj
C:maj
D:7/C
G:maj/B
D:min
E:7/D
A:min/C
A#:(bb3, b5)/C
B:7
E:min/B
B:7
E:min/B
B:7
E:min/B
B:7
E:min/B
B:maj
E:min/B
B:maj
E:min/B
B:maj
E:min/B
B:maj
E:min/B
B:maj
B:7
E:maj
B:7
C#:min
G#:7
A:maj
B:7/F#
E:maj/G#
F#:min7/A
E:maj/B
B:7
E:maj
B:7
C#:min
G#:7
A:maj
E:maj/B
B:7
E:maj
B:7
C#:min
G#:7
A:maj
B:7/F#
E:maj/G#
F#:min7/A
E:maj/B
B:7
E:maj
B:7
C#:min
G#:7
A:maj
E:maj/B
B:7
E:maj
B:7
E:maj/B
B:7
E:maj
B:7/F#
E:maj/G#
B:7/A
E:maj/G#
B:7/D#
E:maj/G#
B:7/F#
E:maj/G#
B:7/F#
E:maj/G#
B:7/F#
E:maj/G#
B:7/F#
E:maj/G#
B:7/F#
E:maj/G#
B:7/A
E:maj/G#
B:7/A
E:maj/G#
B:7/A
E:maj/G#
B:7/A
E:maj/G#
E:7/G#
A:maj
F#:7/A#
E:maj/B
D#:dim7
D#:hdim7
B:7/D#
E:maj
E:7
A:min/E
E:7
A:min
F#:dim/A
E:min/B
B:7
E:min
E:7
A:min/E
E:7
A:min
F#:dim/A
E:min/B
B:7
E:min
E:min/G
F#:dim/A
E:min/B
B:7
C:maj
C:maj/E
D:min/F
C:maj/G
G:7
C:maj
D:7/C
G:maj/B
Bb:maj
C:7